In [211]:
#Dependencies
#pip install xlrd
import numpy as np
import pandas as pd
import datetime as dt
import plotly.express as px
import plotly.subplots as ps
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [212]:
#Read Excel file
df = pd.read_excel('./Resources/The_Numbers_Data.xlsx')

#check right rows to select
# print(df.loc[99:144]) 

#create df
movies_df = df.loc[101:144]

#Labels for columns
movies_df.rename(columns={'The Numbers - Where Data and Movies Meet':'Release_Date',
                          'Unnamed: 1':'Title', 'Unnamed: 2':'Budget','Unnamed: 3':'Opening_Weekend',
                            'Unnamed: 4':'Domestic_Box_Office','Unnamed: 5':'Worldwide_Box_Office'}, inplace = True)
# Drop Werewolf by Night as no numbers along with NaN numbers
movies_df = movies_df.dropna()
# Assign budget for Wakanda Forever
# movies_df.at[114,'Budget']=250000000

# Change to date with datetime
movies_df['Release_Date'] = pd.to_datetime(movies_df['Release_Date'])
# Change date to ascending order
movies_df.sort_values(by='Release_Date', inplace=True)
movies_df

c:\Users\Kevin\anaconda3\envs\bootcamp\lib\site-packages\pandas\core\frame.py:5047: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Release_Date,Title,Budget,Opening_Weekend,Domestic_Box_Office,Worldwide_Box_Office
144,2008-05-02,Iron Man,186000000,102118668,318604126,585171547
143,2008-06-13,The Incredible Hulk,137500000,55414050,134806913,265573859
142,2010-05-07,Iron Man 2,170000000,128122480,312433331,621156389
141,2011-05-06,Thor,150000000,65723338,181030624,449326618
140,2011-07-22,Captain America: The First …,140000000,65058524,176654505,370569776
139,2012-05-04,The Avengers,225000000,207438708,623357910,1515100211
138,2013-05-03,Iron Man 3,200000000,174144585,408992272,1215392272
137,2013-11-08,Thor: The Dark World,150000000,85737841,206362140,644602516
136,2014-04-04,Captain America: The Winter…,170000000,95023721,259746958,714401889
135,2014-08-01,Guardians of the Galaxy,170000000,94320883,333714112,770882395


In [213]:
# create total box office and profit columns
# movies_df['Total_Box_Office'] = movies_df.loc[:,['Domestic_Box_Office','Worldwide_Box_Office']].sum(axis=1)
movies_df = movies_df.assign(International_Box_Office=movies_df['Worldwide_Box_Office'] - movies_df['Domestic_Box_Office'])
movies_df = movies_df.assign(Profit=movies_df['Worldwide_Box_Office'] - movies_df['Budget'])

movies_df.tail()

,Release_Date,Title,Budget,Opening_Weekend,Domestic_Box_Office,Worldwide_Box_Office,International_Box_Office,Profit
116,2022-07-08,Thor: Love and Thunder,250000000,144165107,343256830,760928081,417671251,510928081
114,2022-11-11,Black Panther: Wakanda Forever,250000000,181339761,453829060,853985546,400156486,603985546
113,2023-02-17,Ant-Man and the Wasp: Quant…,200000000,106109650,214506909,463635303,249128394,263635303
112,2023-05-05,Guardians of the Galaxy Vol 3,250000000,118414021,358995815,845468744,486472929,595468744
111,2023-11-10,The Marvels,274800000,46110859,84500223,199706250,115206027,-75093750


In [214]:
# change monetary string values to float values
movies_df = movies_df.astype({'Budget':'float','Opening_Weekend':'float',
                              'Domestic_Box_Office':'float','Worldwide_Box_Office':'float',
                              'International_Box_Office':'float','Profit':'float'})
# set float option to remove sicentific notation
pd.set_option('display.float_format', lambda x: '%.0f' % x)

movies_df.dtypes

Release_Date                datetime64[ns]
Title                               object
Budget                             float64
Opening_Weekend                    float64
Domestic_Box_Office                float64
Worldwide_Box_Office               float64
International_Box_Office           float64
Profit                             float64
dtype: object

In [215]:
fig = go.Figure()
# fig.add_trace(go.Bar(
#     x=movies_df['Title'],
#     y=movies_df['Budget'],
#     name='Budget',
#     marker_color='red'
# ))
fig.add_trace(go.Bar(
    x=movies_df['Title'],
    y=movies_df['Domestic_Box_Office'],
    name='Domestic Box Office',
    marker_color='blue'
))

fig.add_trace(go.Bar(
    x=movies_df['Title'],
    y=movies_df['International_Box_Office'],
    name='International Box Office',
    # marker_color='lightsalmon'
))
fig.update_layout(barmode='group', xaxis_tickangle=-45, title="Domestic vs International Box Office",
                autosize=False,width=1300,height=700)
fig.update_traces(textangle=270)
fig.show()

In [216]:
fig = px.bar(movies_df, y='Budget', x='Title', text_auto='.2s',
            color_discrete_sequence=["red"],
            title="Budget by Movie Title")
fig.update_layout(barmode='group', xaxis_tickangle=-45,
                  autosize=False,width=1300,height=700)
fig.update_traces(textangle=270)

fig.show()

In [217]:
fig = px.bar(movies_df, y='Worldwide_Box_Office', x='Title', text_auto='.2s',
            color_discrete_sequence=["green"],
            title="Worldwide Box Office by Movie Title")
fig.update_layout(barmode='group', xaxis_tickangle=-45,
                  autosize=False,width=1300,height=700)
fig.update_traces(textangle=270)
fig.show()

In [218]:
fig = px.bar(movies_df, x="Title", y=['Budget',"Profit"],  title="Budget vs Profit",
            color_discrete_sequence=["red", "green"],text_auto='0.2s')
fig.update_layout(barmode='stack', xaxis_tickangle=-45,
                autosize=False,width=1300,height=700)
fig.update_traces(textangle=270, textposition='outside', width=0.5)
fig.show()

In [219]:
# Assign MCU phases
# movies_df.reset_index(drop=True, inplace=True)
# movies_df.loc[0:5,'Phase'] = 1
# movies_df.loc[6:11,'Phase'] = 2
# movies_df.loc[12:21,'Phase'] = 3
# movies_df.loc[22:29,'Phase'] = 4
# movies_df.loc[30:,'Phase'] = 5

# Slection of dates for each Phase
# # Select movies in the date range of interest and add a 'Phase' column
# movies_df = (movies_df[(movies_df['Release_Date'] > '2008-03-01') & (movies_df['Release_Date'] <= '2012-05-04')]
#         .assign(Phase=1)
#         .append(movies_df[(movies_df['Release_Date'] > '2013-05-02') & (movies_df['Release_Date'] <= '2015-07-18')]
#         .assign(Phase=2))
#         .append(movies_df[(movies_df['Release_Date'] > '2016-05-03') & (movies_df['Release_Date'] <= '2019-07-02')]
#                 .assign(Phase=3))
#         .append(movies_df[(movies_df['Release_Date'] > '2021-06-09') & (movies_df['Release_Date'] <= '2022-11-11')]
#                 .assign(Phase=4))            
#         .append(movies_df[(movies_df['Release_Date'] > '2023-01-17') & (movies_df['Release_Date'] <= '2024-09-06')]
#                 .assign(Phase=5))
#                 )

# ph1 = movies_df[(movies_df['Release_Date'] > '2008-05-02') & (movies_df['Release_Date'] <= '2012-05-04')]
# Add a new column 'Phase' with a value of 1 for all rows
# ph1 = ph1.assign(Phase=1)
# Initialize 'Phase' column with 0
movies_df['Phase'] = 0

# Assign phases based on release dates
movies_df.loc[(movies_df['Release_Date'] > '2008-03-01') & (movies_df['Release_Date'] <= '2012-05-04'), 'Phase'] = 1
movies_df.loc[(movies_df['Release_Date'] > '2013-05-02') & (movies_df['Release_Date'] <= '2015-07-18'), 'Phase'] = 2
movies_df.loc[(movies_df['Release_Date'] > '2016-05-03') & (movies_df['Release_Date'] <= '2019-07-02'), 'Phase'] = 3
movies_df.loc[(movies_df['Release_Date'] > '2021-06-09') & (movies_df['Release_Date'] <= '2022-11-11'), 'Phase'] = 4
movies_df.loc[(movies_df['Release_Date'] > '2023-01-17') & (movies_df['Release_Date'] <= '2024-09-06'), 'Phase'] = 5

movies_df.to_csv('marvel_box_office.csv', sep='\t', encoding='utf-8')
movies_df


,Release_Date,Title,Budget,Opening_Weekend,Domestic_Box_Office,Worldwide_Box_Office,International_Box_Office,Profit,Phase
144,2008-05-02,Iron Man,186000000,102118668,318604126,585171547,266567421,399171547,1
143,2008-06-13,The Incredible Hulk,137500000,55414050,134806913,265573859,130766946,128073859,1
142,2010-05-07,Iron Man 2,170000000,128122480,312433331,621156389,308723058,451156389,1
141,2011-05-06,Thor,150000000,65723338,181030624,449326618,268295994,299326618,1
140,2011-07-22,Captain America: The First …,140000000,65058524,176654505,370569776,193915271,230569776,1
139,2012-05-04,The Avengers,225000000,207438708,623357910,1515100211,891742301,1290100211,1
138,2013-05-03,Iron Man 3,200000000,174144585,408992272,1215392272,806400000,1015392272,2
137,2013-11-08,Thor: The Dark World,150000000,85737841,206362140,644602516,438240376,494602516,2
136,2014-04-04,Captain America: The Winter…,170000000,95023721,259746958,714401889,454654931,544401889,2
135,2014-08-01,Guardians of the Galaxy,170000000,94320883,333714112,770882395,437168283,600882395,2


In [226]:
# Split the DataFrame into separate DataFrames for each phase
phase_dfs = {}
for phase in range(1, 6):
    phase_dfs[phase] = movies_df[movies_df['Phase'] == phase].sort_index(ascending=False).copy()


phase_1 = phase_dfs[1]
phase_2 = phase_dfs[2]
phase_3 = phase_dfs[3]
phase_4 = phase_dfs[4]
phase_5 = phase_dfs[5]

print(phase_1)

    Release_Date                         Title    Budget  Opening_Weekend  \
144   2008-05-02                      Iron Man 186000000        102118668   
143   2008-06-13           The Incredible Hulk 137500000         55414050   
142   2010-05-07                    Iron Man 2 170000000        128122480   
141   2011-05-06                          Thor 150000000         65723338   
140   2011-07-22  Captain America: The First … 140000000         65058524   
139   2012-05-04                  The Avengers 225000000        207438708   

     Domestic_Box_Office  Worldwide_Box_Office  International_Box_Office  \
144            318604126             585171547                 266567421   
143            134806913             265573859                 130766946   
142            312433331             621156389                 308723058   
141            181030624             449326618                 268295994   
140            176654505             370569776                 193915271   
139 

In [221]:
# Convert 'Profit' column to numeric
phase_1['Profit'] = phase_1['Profit'].astype(float)

# Scatter plot DataFrame containing Phase One movies
fig_p1 = px.scatter(phase_1, x=phase_1.index, y="Worldwide_Box_Office", size='Profit',
                    trendline="ols", trendline_scope="overall",
                    color_discrete_sequence=["blue", "red"],
                    title="Phase One Worldwide Box Office by Movie Title")

fig_p1.update_layout(
    autosize=False,
    width=800,
    height=500,
    margin=dict(),
    xaxis=dict(
        tickmode='array',
        tickvals=list(phase_1.index),
        ticktext=phase_1['Title'].tolist(),
        title='Movie Title'
    )
)

fig_p1.show()


In [227]:
# Convert 'Profit' column to numeric
phase_2['Profit'] = phase_2['Profit'].astype(float)

# Scatter plot DataFrame containing Phase One movies
fig_p2 = px.scatter(phase_2, x=phase_2.index, y="Worldwide_Box_Office", size='Profit',
                     trendline="ols", trendline_scope="overall",
                     color_discrete_sequence=["orange", "red"],
                     title="Phase Two Worldwide Box Office by Movie Title")

fig_p2.update_layout(
    autosize=False,
    width=800,
    height=500,
    margin=dict(),
    xaxis=dict(
        tickmode='array',
        tickvals=list(phase_2.index),
        ticktext=phase_2['Title'].tolist(),
        title='Movie Title'
    )
)

fig_p2.show()



In [223]:
# Convert 'Profit' column to numeric
phase_3['Profit'] = phase_3['Profit'].astype(float)

# Scatter plot DataFrame containing Phase One movies
fig_p3 = px.scatter(phase_3, x=phase_3.index, y="Worldwide_Box_Office", size='Profit',
                     trendline="ols", trendline_scope="overall",
                     color_discrete_sequence=["green", "red"],
                     title="Phase Three Worldwide Box Office by Movie Title")

fig_p3.update_layout(
    autosize=False,
    width=800,
    height=500,
    margin=dict(),
    xaxis=dict(
        tickmode='array',
        tickvals=list(phase_3.index),
        ticktext=phase_3['Title'].tolist(),
        title='Movie Title'
    )
)

fig_p3.show()


In [224]:
phase_4['Profit'] = phase_4['Profit'].astype(float)

# Scatter plot DataFrame containing Phase One movies
fig_p4 = px.scatter(phase_4, x=phase_4.index, y="Worldwide_Box_Office", size='Profit',
                     trendline="ols", trendline_scope="overall",
                     color_discrete_sequence=["red", "red"],
                     title="Phase Four Worldwide Box Office by Movie Title")

fig_p4.update_layout(
    autosize=False,
    width=800,
    height=500,
    margin=dict(),
    xaxis=dict(
        tickmode='array',
        tickvals=list(phase_4.index),
        ticktext=phase_4['Title'].tolist(),
        title='Movie Title'
    )
)

fig_p4.show()

In [225]:
# Convert 'Profit' column to numeric
phase_5['Profit'] = phase_5['Profit'].astype(float)

# Scatter plot DataFrame containing Phase One movies
fig_p5 = px.scatter(phase_5, x=phase_5.index, y="Worldwide_Box_Office", size='Profit',
                     trendline="ols", trendline_scope="overall",
                     color_discrete_sequence=["purple", "red"],
                     title="Phase Five Worldwide Box Office by Movie Title")

fig_p5.update_layout(
    autosize=False,
    width=800,
    height=500,
    margin=dict(),
    xaxis=dict(
        tickmode='array',
        tickvals=list(phase_5.index),
        ticktext=phase_5['Title'].tolist(),
        title='Movie Title'
    )
)

fig_p5.show()

ValueError: 
    Invalid element(s) received for the 'size' property of scatter.marker
        Invalid elements include: [-75093750.0]

    The 'size' property is a number and may be specified as:
      - An int or float in the interval [0, inf]
      - A tuple, list, or one-dimensional numpy array of the above

In [ ]:
# use facet_col to show phases and trendlines in one chart
fig = px.scatter(movies_df, x=movies_df.index, y="Worldwide_Box_Office", size="Worldwide_Box_Office", #change size to profit once Marvels becomes positive.
                facet_col="Phase", color='Phase', trendline="ols",)
                
fig.update_xaxes(matches=None)
fig.update_xaxes(tickangle=45, title="Movie Title")
fig.update_yaxes(title="Worldwide Box Office")
fig.for_each_xaxis(lambda xaxis: xaxis.update(showticklabels=True))
fig.update_layout(height=500, width=1200,
                  xaxis = dict(
                    tickmode = 'array',
                    tickvals = [0,1, 2, 3, 4],
                    ticktext = ['Iron Man', 'The Incredible Hulk', 'Thor', "Iron Man 2",
                    'Captain America',"The Avengers"]),
                  xaxis2 = dict(
                    tickmode = 'array',
                    tickvals = [5,6,7, 8, 9, 10],
                    ticktext = ['Iron Man 3', 'Thor: The Dark World',
                    'Captain America Winter Soldier','Guardians of the Galaxy',
                    "Avengers: Age of Ultron", "Ant Man"]),
                  xaxis3 = dict(
                      tickmode = 'array',
                    tickvals = [11,12,13,14,15,16,17,18,19,20,21],
                    ticktext = ['Captain America Civil War', 'Doctor Strange',
                    'Guardians of the Galaxy Vol 2', "Spider-Man: Homecoming",
                    'Thor: Ragnarok',"Black Panther	", 'Avengers: Infinity War',"Ant-Man and the Wasp",
                    'Captain Marvel', 'Avengers: Endgame',"Spider-Man: Far from Home" ]),
                  xaxis4 = dict(
                    tickmode = 'array',
                    tickvals = [22,23,24,25,26,27,28],
                    ticktext = [ 'Black Widow', 'Shang-Chi', 'Eternals',
                                'Spider-Man: No Way Home', "Doctor Strange: MOM",'Thor: Love and Thunder',
                                "Black Panther: Wakanda Forever"]),
                   
                   xaxis5=dict(
                    tickmode = 'array',
                    tickvals = [29,30,31],
                    ticktext=[ "Ant-Man and the Wasp: Quantumania", "Guardians Vol 3", "The Marvels"]
                  ))


fig.show()




In [ ]:
ph5 = movies_df[(movies_df['Release_Date'] > '2023-01-17') & (movies_df['Release_Date'] <= '2024-09-06')]

# scatter subplots for ww box office profit
fig = make_subplots(rows=3, cols=2, start_cell="top-left",
                    subplot_titles=("Phase 1", "Phase 2", "Phase 3", "Phase 4", 'Phase 5'))
                    

fig.add_trace(go.Scatter(x=ph1.index, y=ph1['Profit'],
                 name="Phase 1"),
              row=1, col=1)

fig.add_trace(go.Scatter(x=ph2.index, y=ph2['Profit'],
               name="Phase 2"),
              row=1, col=2)

fig.add_trace(go.Scatter(x=ph3.index, y=ph3['Profit'],
               name="Phase 3"),
              row=2, col=1)

fig.add_trace(go.Scatter(x=ph4.index, y=ph4['Profit'],
               name="Phase 4"),
              row=2, col=2)
fig.add_trace(go.Scatter(x=ph5.index, y=ph5['Profit'],
               name="Phase 5"),
              row=3, col=1)

fig.update_layout(height=1000, width=1000,
                  xaxis = dict(
                    tickmode = 'array',
                    tickvals = [0,1, 2, 3, 4],
                    ticktext = ['Iron Man', 'The Incredible Hulk', 'Thor', "Iron Man 2",
                    'Captain America',"The Avengers"]),
                  xaxis2 = dict(
                    tickmode = 'array',
                    tickvals = [5,6,7, 8, 9, 10,],
                    ticktext = ['Iron Man 3', 'Thor: The Dark World',
                    'Captain America Winter Soldier','Guardians of the Galaxy',
                    "Avengers: Age of Ultron", "Ant Man"]),
                  xaxis3 = dict(
                      tickmode = 'array',
                    tickvals = [11,12,13,14,15,16,17,18,19,20,21],
                    ticktext = ['Captain America Civil War', 'Doctor Strange',
                    'Guardians of the Galaxy Vol 2', "Spider-Man: Homecoming",
                    'Thor: Ragnarok',"Black Panther	", 'Avengers: Infinity War',"Ant-Man and the Wasp",
                    'Captain Marvel', 'Avengers: Endgame',"Spider-Man: Far from Home" ]),
                  xaxis4=dict(
                      tickmode = 'array',
                      tickvals = [22,23,24,25,26,27,28,29],
                      ticktext = [ 'Black Widow', 'Shang-Chi', 'Eternals',
                                'Spider-Man: No Way Home', "Doctor Strange: MOM",'Thor: Love and Thunder',
                                "Black Panther: Wakanda Forever"]),
                    xaxis5=dict(
                    tickmode = 'array',
                    tickvals = [30,31,32],
                    ticktext=[ "Ant-Man and the Wasp: Quantumania", "Guardians Vol 3", "The Marvels"]),

                  title_text="Profit of each Movie by Phase")
fig.update_xaxes(tickangle=45)
fig.show()

In [ ]:
# Group and average phases
four_phase_avg = movies_df.groupby(['Phase']).mean()
four_phase_avg =four_phase_avg.reset_index()
#Remove Spiderman moviesfrom phase 4 and average phases
no_spider2 = movies_df.drop([22, 26])
no_spider_avg = no_spider2.groupby(['Phase']).mean()
no_spider_avg = no_spider_avg.reset_index()

In [ ]:
# Show profitabilty of all 4 phases
fig = px.scatter(four_phase_avg, x='Phase', y="Profit", size = 'Profit',trendline="ols", trendline_scope="overall")
fig.update_layout(
            autosize=False,
    width=1000,
    height=500,
    margin=dict(
    ),
    xaxis = dict(
        tickmode = 'array',
        tickvals = [1, 2, 3, 4,5],
        ticktext = ['One', 'Two', 'Three', 'Four', 'Five']
    )
)
fig.show()

In [ ]:


labels = ['Phase 1', 'Phase 2', 'Phase 3', 'Phase 4', 'Phase 5']

# Define color sets of paintings
colors = px.colors.sequential.RdBu
# Create subplots, using 'domain' type for pie charts
specs = [[{'type':'domain'}, {'type':'domain'}], [{'type':'domain'}, {'type':'domain'}]]
fig = make_subplots(rows=2, cols=2, specs=specs,
                     subplot_titles=['Worldwide Box Office',
                    'Worldwide Box Office No Spiderman', "Profit Phase", "Profit no Spiderman"])

# Define pie charts
fig.add_trace(go.Pie(labels=labels, values=four_phase_avg['Worldwide_Box_Office'],
                     name='With Spiderman',marker_colors=colors), 1, 1)
fig.add_trace(go.Pie(labels=labels, values=no_spider_avg['Worldwide_Box_Office'],
                     name='No Spiderman',
                     marker_colors=colors), 1, 2)
fig.add_trace(go.Pie(labels=labels, values=four_phase_avg['Profit'],
                     name='With Spiderman',marker_colors=colors), 2, 1)
fig.add_trace(go.Pie(labels=labels, values=no_spider_avg['Profit'],
                     name='No Spiderman',
                     marker_colors=colors), 2, 2)

# Tune layout and hover info
fig.update_traces(hoverinfo='label+percent+name')
fig.update(layout_title_text='Worldwide Box Office and Profit Phase',
           layout_showlegend=True,)
fig.update_layout(autosize=False,width=1000,height=500)

# fig = go.Figure(fig)
fig.show()